# Isochronen in Freiberg 3
## Dominos Pizza als Beispiel

Erreichbarkeitsanalyse mit OpenStreetMap Daten für den Dominos Pizza in Freiberg.

Dominos OSM ID 4750837413 https://www.openstreetmap.org/node/4750837413

Die nächste Kreuzung wird vom Tool als Startpunkt genommen. ID 2487859077 https://www.openstreetmap.org/node/2487859077

Dargestellt werden verschiedene Isochronen für verschiedene erlaubte Straßen/Wege.

- Die Fahzeit beträgt jeweils 15 Minuten.
- Höchste erlaubte Geschwindigkeit in der Stadt: 6km/h = 1.66m/s
- Betriebsgeschwindigkeit unserer aktuellen Plattform: 3.6km/h = 1m/s
- Die Isochronen wurden für die Betriebsgeschwindigkeit berechnet.

Eingezeichnet sind auch:

- Maximale Lieferdistanz von Dominos 2,6 km (großer roter Kreis)
- Maximale Distanz die in 15 Minuten zurückgelegt werden kann: 900m (= 1m/s * 15min * 60s/min) (kleiner grüner Kreis)

Die Karte ist interaktiv. Oben rechts können verschiedene Layer ein- und ausgeschaltet werden.

Die Straßen können angeklickt werden um die Namen und vorhandenen OSM Attribute anzuzeigen.

### Scenario Beschreibungen

- Small: Nur Gehwege und Fußgängerzonen sind erlaubt.
- Medium: Alles außer große Straßen sind erlaubt.
- Medium-restricted: Geh- und Radwege sind erlaubt.
- Large: Nur Straßen sind erlaubt, keine Fuß und Radwege.
- Optimal: Vergleichsszenario, alles ist erlaubt.

In [ ]:
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import osmnx as ox
from configobj import ConfigObj
from descartes import PolygonPatch
from helpers import configreader as cfr
from helpers import osmhelper as oh
from helpers import ready4robots as r4r
from numpy import asarray, concatenate, ones
from pyproj import Proj, transform
from shapely.geometry import LineString, MultiPolygon, Point, Polygon
from shapely.geometry import mapping

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
cfg = ConfigObj("notebook.ini")
place = "Freiberg, Sachsen"
trip_times = [5, 10, 15, 20, 25, 30]  # in minutes
travel_speed = 3.6  # in km/h

ox.config(useful_tags_way=cfr.get_relevant_tags(cfg))
ox.config(log_console=True, use_cache=True)
ox.__version__

In [ ]:
# download the street network
G_84 = r4r.get_graph(place)

In [ ]:
center_node = ox.nearest_nodes(G_84, 13.3447448, 50.9162500)
# center_node = 3930896811
G = ox.project_graph(G_84)

In [ ]:
center_node

In [ ]:
# add an edge attribute for time in minutes required to traverse each edge
meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
for u, v, k, data in G.edges(data=True, keys=True):
    data['time'] = data['length'] / meters_per_minute

In [ ]:
def convert_coordinates(geom):
    """Convert UTM Polygon/MultiPolygon to Lat/Lon Polygon/MultiPolygon."""

    # Define the UTM zone projection and the lat/long projection
    utm_zone = "EPSG:32633"  # Example for UTM zone 33N, change as per your UTM zone
    latlong = "EPSG:4326"
    # Change zone accordingly
    in_proj = Proj(proj='utm', zone=33, datum='WGS84')
    out_proj = Proj(proj='latlong', datum='WGS84')

    if geom.geom_type == 'Polygon':
        exterior_coords = [transform(in_proj, out_proj, x, y)
                           for x, y in geom.exterior.coords]
        interior_coords = [[transform(in_proj, out_proj, x, y)
                            for x, y in interior.coords] for interior in geom.interiors]
        return Polygon(exterior_coords, holes=interior_coords)
    elif geom.geom_type == 'MultiPolygon':
        polygons = []
        for polygon in geom:
            polygons.append(convert_coordinates(polygon))
        return MultiPolygon(polygons)
    else:
        raise ValueError(f"Unsupported geometry type: {geom.geom_type}")


# converted_isochrone_polys = [convert_coordinates(
#     polygon) for polygon in isochrone_polys]

In [ ]:
def prep_graph(place, szenario, travel_speed=3.6):

    G_84 = r4r.get_graph(place)

    scenario_filter = r4r.get_scenario_filter(cfg, szenario["name"])
    H_84 = oh.filter_graph_by_dict(
        G_84.copy(), scenario_filter, drop_nodes=False)

    center_node = ox.nearest_nodes(H_84, 13.3447448, 50.9162500)
    H = ox.project_graph(H_84)

    # add an edge attribute for time in minutes required to traverse each edge
    meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
    for u, v, k, data in H.edges(data=True, keys=True):
        data['time'] = data['length'] / meters_per_minute

    return H, H_84

In [ ]:
def make_iso_polys_time(G, edge_buff=25, node_buff=50, infill=False, time_window=15):
    # Assume trip_times is a list of time values (in minutes) you're interested in
    trip_times = [time_window]

    isochrone_polys = []

    for trip_time in trip_times:
        subgraph = nx.ego_graph(
            G, center_node, radius=trip_time, distance='time')

        node_points = [Point((data['x'], data['y']))
                       for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame(
            {'id': subgraph.nodes()}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index('id')

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to).get(0,  None)
            if edge_lookup is None:
                continue
            edge_lookup = edge_lookup.get(
                'geometry',  LineString([f, t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union

        if infill:
            new_iso = Polygon(new_iso.exterior)

        isochrone_polys.append(new_iso)

    return isochrone_polys

In [ ]:
szenarios = list(cfr.yield_scenarios(cfg))
H1, _ = prep_graph(place, szenarios[0], travel_speed)
H2, _ = prep_graph(place, szenarios[1], travel_speed)
H3, _ = prep_graph(place, szenarios[2], travel_speed)
H4, _ = prep_graph(place, szenarios[3], travel_speed)
H5, _ = prep_graph(place, szenarios[4], travel_speed)

In [ ]:
# sorting just to show the smaller once on top.
subgraphs = [H5, H2, H4, H3, H1]
colors = ['red', 'blue', 'green', 'yellow', 'purple']

fg_H5 = folium.FeatureGroup(name='Optimal')
fg_H2 = folium.FeatureGroup(name='Medium')
fg_H4 = folium.FeatureGroup(name='Large')
fg_H3 = folium.FeatureGroup(name='Medium-restricted')
fg_H1 = folium.FeatureGroup(name='Small')

fgs = [fg_H5, fg_H2, fg_H4, fg_H3, fg_H1]

m = folium.Map(location=[50.915066, 13.346332], zoom_start=15)
m = ox.graph_to_gdfs(G_84, nodes=False).explore(m=m)


for i, (subgraph, fg) in enumerate(zip(subgraphs, fgs)):
    # Generate 15-min isochrone
    iso_polys = make_iso_polys_time(
        subgraph, edge_buff=25, node_buff=0, infill=True, time_window=15)

    # Convert coordinates if needed
    converted_iso_polys = [convert_coordinates(
        polygon) for polygon in iso_polys]

    # Add to Folium map
    for poly in converted_iso_polys:
        folium.GeoJson(
            data=mapping(poly),
            style_function=lambda x, color=colors[i]: {
                'color': color,
                'weight': 5,
                'fill': False,
                # 'fillColor': color,
                # 'fillOpacity': 0.1
            }
        ).add_to(fg)
    fg.add_to(m)

folium.LayerControl().add_to(m)

folium.Circle(
    location=[50.915066, 13.346332],
    radius=900,  # radius in meters
    color='green',
    fill=False,
    # fill_color='blue',
    # fill_opacity=0.05
).add_to(m)

folium.Circle(
    location=[50.915066, 13.346332],
    radius=2600,  # radius in meters
    color='blue',
    fill=False,
    # fill_color='blue',
    # fill_opacity=0.05
).add_to(m)

# Show the map
# m.save("dominos_map_full.html")

In [ ]:
m